In [ ]:
#Install selenium with 

In [11]:
import sys
import wget
import newspaper
from newspaper import Article
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
import itertools
import lxml
import xmltodict
import collections
from urllib import request
from collections import OrderedDict
from urllib.error import HTTPError, URLError
from urllib.parse import urlparse
import sys
import ast
import time
from pandarallel import pandarallel
import requests
import datefinder
from datetime import datetime
from tqdm import tqdm
import psutil
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from multiprocessing import Pool
import random



In [9]:
#PLEASE EDIT THIS CELL!

#Universal parameters. Always needs to be changed
news_outlet = "elespectador"
country = "colombia"
max_click_SHOW_MORE = 500
host = "https://www.elespectador.com/"
date_format = "%Y-%m-%d"
relevant_path = f'^{host}.*$'
urls = ['archivo/politica/','archivo/mundo/','archivo/judicial/']
urls = [host + url for url in urls]
csv_name = country+'_'+news_outlet+'.csv'
types = ['page', 'click_more','scroll_down']
type_of_page = types[0]
extract_text_sleep = random.randint(4,6)
extract_urls_sleep = random.randint(2,4)





# If page type is click_more or scroll_down i.e. can get more article by clicking a button with text 'click more' or simply scrolling down
xpath_for_link = "//h2[contains(@class, 'Card-Title Title Title')]//following::a"
xpath_for_click_more = "//div[contains(text(), 'Cargar Más')]"





# If page type is page
target_tag = 'h2'
target_tag_class = 'Card-Title Title Title'

if type_of_page == 'page':
    urls = [url for url in urls] #   #   + 'page/' #when switching between pages like for example from page 1 to 2, 
                                            # the url could switch from either nacionales/1/ to nacionales/2/ or it could be 
                                            # nacionales/page/1/ to nacionales/page/2/. In that case the statement 
                                            # here will be 
                        
                                            # urls = [url + 'page/' for url in urls]
                        
                                            #Please look at the url change in the website and make the modification here
pages_each_category = [834,834,834] #The total number of pages for each section. So if policiaca is 1..92, write 92, not 93.
if pages_each_category:
    total_pages = pages_each_category
else:
    total_pages = 500

In [3]:
def parallelize_dataframe(df, func, n_cores):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [4]:
output_path = 'links_'+csv_name

def retrieve_links_from_list(url,scroll_down=False):
  options = FirefoxOptions()
  options.add_argument("--headless")
  driver = webdriver.Firefox(options=options)
  driver.get(url)
  
  count = 0

  news_urls = set()

  prev_length = 0
  while count <= max_click_SHOW_MORE:
      if not scroll_down:
          try:
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, xpath_for_click_more))
            )
          except TimeoutException as e:
            print('Could not find click more button')
            break
        
    
      try:
          all_links = driver.find_elements(By.XPATH, xpath_for_link)
          if not scroll_down:
            click_more = driver.find_element(By.XPATH, xpath_for_click_more)
          curr_length = len(all_links)
          print(f'{curr_length} links in current page')
          print(f'{prev_length} links in previous page')
          if count > 0:
            if curr_length == prev_length or (not scroll_down and not click_more):
              break
          
          current_links = [l.get_attribute("href") for l in all_links[prev_length:]]
          df_link = pd.DataFrame(current_links, columns = ['link'])
          df_link.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
            
          prev_length = curr_length

          if not scroll_down:
            
            try:
                click_more.click();
            except Exception as e:
                driver.execute_script("arguments[0].click();", click_more) #If click does not work because of overlapping elements, this executes
            
            print(f"Button clicked {count} times", )
          else:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            print(f"Scrolled down {count+1} times")
          
          time.sleep(3)
            
          count += 1
        
          
          
      except TimeoutException:
          break

  time.sleep(2)

  all_links = driver.find_elements(By.XPATH, xpath_for_link)

  for link in all_links:
        if isinstance(link.get_attribute('href'), str):
          if re.match(relevant_path, link.get_attribute('href')):
            news_urls.add(link.get_attribute('href'))

  driver.quit()
  print('*' * 20)
  return list(news_urls)


def extract_urls(url):    
    rows = set()
    soup = getSoup(url)
    if soup:
      all_elems = None
      if target_tag_class != '':
        all_elems = soup.find_all(target_tag, {'class':target_tag_class})
      else:
        all_elems = soup.find_all(target_tag)
      
      for d in all_elems:
        all_links = d.find_all('a', href=True)
        
        for l in all_links:
            if not re.match('^http.*', l['href']):
              l['href'] = host + l['href']
              
            if re.match(relevant_path, l['href']):
              rows.add(l['href'])
                
    
    if rows:
      return list(rows)
    else:
      return None

def extract_urls_from_df(df):
    links = df['page'].map(lambda x: extract_urls(x))
    df['link'] = links
    return df

In [14]:
pages = []

def get_zipped_urls(url, total_pages):
    page_numbers = [str(i) for i in range(1,total_pages)]
    url_multiple = [url] * total_pages
    return [''.join(x) for x in zip(url_multiple,page_numbers)]

if type_of_page == 'page':
    for ind, url in enumerate(urls):
        if isinstance(total_pages, int):
            pages += get_zipped_urls(url, total_pages)
        else:
            pages_for_category = total_pages[ind] + 1
            pages += get_zipped_urls(url, pages_for_category)
            
elif type_of_page == 'click_more' or type_of_page == 'scroll_down':
    for url in urls:
        pages += retrieve_links_from_list(url, True if type_of_page == 'scroll_down' else False)

        
df = pd.DataFrame(pages, columns = ['page'])
df

,page
0,https://www.elespectador.com/archivo/politica/1
1,https://www.elespectador.com/archivo/politica/2
2,https://www.elespectador.com/archivo/politica/3
3,https://www.elespectador.com/archivo/politica/4
4,https://www.elespectador.com/archivo/politica/5
...,...
2497,https://www.elespectador.com/archivo/judicial/830
2498,https://www.elespectador.com/archivo/judicial/831
2499,https://www.elespectador.com/archivo/judicial/832
2500,https://www.elespectador.com/archivo/judicial/833


In [15]:
def getSoup(url):
    """
    Return a soup object of the URL
    """
    try:
        req = request.Request(url, headers={'User-Agent' : "Chrome"}) 
        con = request.urlopen(req)
        time.sleep(extract_urls_sleep)
        html = con.read()

    except HTTPError as e:
        print(e)
        return None
        
    except URLError as e:
        print('The server could not be found')
        return None

    except Exception as e:
      print(e)
      return None

    
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [16]:
def extract_text(url):
    text, title, date = None, None, None
    try:
        result = [country, news_outlet] + [''] * 3 
        article = Article(url, keep_article_html=False)
        article.download()
        time.sleep(extract_text_sleep)
        article.parse()
        text = article.text
        
        if text:
            text_copy = text
            title = article.title
            date = article.publish_date
            if date:
                date = date.strftime(date_format)
            else:
                matches = datefinder.find_dates(text_copy)
                most_recent_datetime = sorted(matches)[-1]
                date = most_recent_datetime.strftime(date_format)
    finally:
      if title:
        result[2] = title
      if date:
        result[3] = date
      if text:
        result[4] = text
      return result 


def extract_text_from_df(df):
    content = df['link'].map(lambda x: extract_text(x))
    df['content'] = content
    df[['country', 'news_outlet', 'title', 'date', 'text']] = pd.DataFrame(df.content.tolist(), index= df.index)
    df = df.drop(['content'], axis=1)
    if 'Unnamed: 0' in df:
        df = df.drop(['Unnamed: 0'], axis=1)
    
    return df

In [ ]:
df

,page
0,https://www.elespectador.com/archivo/politica/page/1
1,https://www.elespectador.com/archivo/politica/page/2
2,https://www.elespectador.com/archivo/politica/page/3
3,https://www.elespectador.com/archivo/politica/page/4
4,https://www.elespectador.com/archivo/politica/page/5
...,...
2497,https://www.elespectador.com/archivo/judicial/page/830
2498,https://www.elespectador.com/archivo/judicial/page/831
2499,https://www.elespectador.com/archivo/judicial/page/832
2500,https://www.elespectador.com/archivo/judicial/page/833


In [17]:
def parallel_work(df, method_to_run, target, source):
  global workers
  while workers>0:
    try:
      df[target] = df[source].parallel_apply(method_to_run)
    except Exception as e:  
      raise e
    break

  if workers == 0:
    print('Error during parallel operation. Could not extract text')

  return df

def get_parallel_operation_results(divided_dfs, method_to_run, target, source):
  res = []
  for df in divided_dfs:
    try:
      temp_df = parallel_work(df, method_to_run, target, source)
      if temp_df[target].isnull().all():
        print('Could not retrieve any URLS')
        print('Something is wrong with the target_tag and target_tag_class variables. Please modify')
        return []
      res.append(temp_df)
    except Exception:
      continue
  df_result = pd.concat(res)
  return df_result

def partition_df(df):
  global articles_per_parallel_operation
  divided_dfs = []
  start = 0
  while start < len(df):
    divided_dfs.append(df[start:start+articles_per_parallel_operation])
    start += articles_per_parallel_operation
  return divided_dfs

In [18]:
if type_of_page == 'page':    
    start = 0
    limit = 40
    total_time_start = time.time()
    results = []
    
    while start < len(df):
        start_time = time.time()
        results.append(parallelize_dataframe(df[start:start+limit], extract_urls_from_df, 20))
        end_time = time.time()
        print(f'Batch of data of row range {start}-{start+limit} complete in {round(end_time-start_time, 2)} seconds')
        print(f'{round(min((((start+limit) / len(df)) * 100), 100), 2)}% complete')
        start+=limit
    
    df = pd.concat(results)
    total_time_end = time.time()
    print(f'total time taken: {round(total_time_end - total_time_start,2)} second')

else:
    df = df.rename(columns={"page": "link"})

In [11]:
df

,link
0,https://www.elespectador.com/politica/cne-abre-indagacion-preliminar-por-los-denominados-petrovideos/
1,https://www.elespectador.com/judicial/defensoria-presenta-proyecto-de-ley-para-el-reconocimiento-de-las-comunidades-afro/
2,https://www.elespectador.com/judicial/defensoria-presenta-proyecto-de-ley-para-el-reconocimiento-de-las-comunidades-afro/#comments
3,https://www.elespectador.com/politica/cne-abre-indagacion-preliminar-por-los-denominados-petrovideos/#comments
4,https://www.elespectador.com/archivo/politica/2/
5,https://www.elespectador.com/politica/asi-estaran-conformadas-las-comisiones-de-senado-y-camara-2022-2026/#comments
6,https://www.elespectador.com/politica/mariposa-recien-descubierta-sera-bautizada-con-el-apellido-de-ivan-duque/#comments
7,https://www.elespectador.com/politica/mariposa-recien-descubierta-sera-bautizada-con-el-apellido-de-ivan-duque/
8,https://www.elespectador.com/mundo/mas-paises/terremoto-de-magnitud-71-sacude-el-norte-de-filipinas/#comments
9,https://www.elespectador.com/mundo/america/maduro-insiste-en-supuestas-amenazas-terroristas-desde-bogota-contra-venezuela/#comments


In [12]:
df.to_csv('links_'+csv_name)

In [13]:
print(len(df))
df = df[df['link'].notna()]
df = df[(df['link'].str.len() != 0)]
print(len(df))

24
24


In [14]:
if type_of_page == 'page':
    df = df.explode('link')
    
df

,link
0,https://www.elespectador.com/politica/cne-abre-indagacion-preliminar-por-los-denominados-petrovideos/
1,https://www.elespectador.com/judicial/defensoria-presenta-proyecto-de-ley-para-el-reconocimiento-de-las-comunidades-afro/
2,https://www.elespectador.com/judicial/defensoria-presenta-proyecto-de-ley-para-el-reconocimiento-de-las-comunidades-afro/#comments
3,https://www.elespectador.com/politica/cne-abre-indagacion-preliminar-por-los-denominados-petrovideos/#comments
4,https://www.elespectador.com/archivo/politica/2/
5,https://www.elespectador.com/politica/asi-estaran-conformadas-las-comisiones-de-senado-y-camara-2022-2026/#comments
6,https://www.elespectador.com/politica/mariposa-recien-descubierta-sera-bautizada-con-el-apellido-de-ivan-duque/#comments
7,https://www.elespectador.com/politica/mariposa-recien-descubierta-sera-bautizada-con-el-apellido-de-ivan-duque/
8,https://www.elespectador.com/mundo/mas-paises/terremoto-de-magnitud-71-sacude-el-norte-de-filipinas/#comments
9,https://www.elespectador.com/mundo/america/maduro-insiste-en-supuestas-amenazas-terroristas-desde-bogota-contra-venezuela/#comments


In [15]:
print(len(df))
df = df.drop_duplicates(subset='link')
print(len(df))

24
22


In [16]:
#Create a links checkpoint csv
df.to_csv('links_'+csv_name)

In [17]:
if 'df' not in locals() or df is None or 'link' not in df:
    if(os.path.exists('links_'+csv_name)):
        df = pd.read_csv('links_'+csv_name, index_col=0)

In [18]:
df

,link
0,https://www.elespectador.com/politica/cne-abre-indagacion-preliminar-por-los-denominados-petrovideos/
1,https://www.elespectador.com/judicial/defensoria-presenta-proyecto-de-ley-para-el-reconocimiento-de-las-comunidades-afro/
2,https://www.elespectador.com/judicial/defensoria-presenta-proyecto-de-ley-para-el-reconocimiento-de-las-comunidades-afro/#comments
3,https://www.elespectador.com/politica/cne-abre-indagacion-preliminar-por-los-denominados-petrovideos/#comments
4,https://www.elespectador.com/archivo/politica/2/
5,https://www.elespectador.com/politica/asi-estaran-conformadas-las-comisiones-de-senado-y-camara-2022-2026/#comments
6,https://www.elespectador.com/politica/mariposa-recien-descubierta-sera-bautizada-con-el-apellido-de-ivan-duque/#comments
7,https://www.elespectador.com/politica/mariposa-recien-descubierta-sera-bautizada-con-el-apellido-de-ivan-duque/
8,https://www.elespectador.com/mundo/mas-paises/terremoto-de-magnitud-71-sacude-el-norte-de-filipinas/#comments
9,https://www.elespectador.com/mundo/america/maduro-insiste-en-supuestas-amenazas-terroristas-desde-bogota-contra-venezuela/#comments


In [19]:
output_path= 'unprocessed_' + csv_name
start = 0
limit = 40

res = []
while start < len(df):
    start_time = time.time()
    demo_df = df[start:start+limit].copy()
    test_df = parallelize_dataframe(demo_df, extract_text_from_df, 20)
    res.append(test_df)
    test_df.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
    
#     test_df['text_set'] = test_df['content'].map(lambda x: True if x[4] != '' else False)

    completed = test_df[test_df.text != ''].shape[0]
#     no_of_text_retrieved = test_df.text_set.sum()
#     print(f'{no_of_text_retrieved} / {len(test_df)} text retrieved')
    print(f'{completed} / {len(test_df)} text retrieved')
    
    end_time = time.time()
    print(f'Batch of data of row range {start}-{start+limit} complete in {round(end_time-start_time, 2)} seconds')
    print(f'{round(min((((start+limit) / len(df)) * 100), 100), 2)}% complete')
    start+=limit

df = pd.concat(res)

In [9]:
output_path= 'unprocessed_' + csv_name
if 'df' not in locals() or df is None or 'content' not in df:
    if(os.path.exists(output_path)):
#         df = pd.read_csv(output_path, on_bad_lines='skip', converters={'content': pd.eval}, index_col=0)
        df = pd.read_csv(output_path, on_bad_lines='skip', index_col=0)





In [10]:
# df[['country', 'news_outlet', 'title', 'date', 'text']] = pd.DataFrame(df.content.tolist(), index= df.index)     
# df

,Unnamed: 0.1,Unnamed: 0,page,link,content,country,news_outlet,title,date,text
0,0,0,https://www.quienopina.com/category/noticias-nacionales/page/1,https://www.quienopina.com/2022/07/hoy-mas-que-nunca-es-necesaria-una-cicih-luis-javier-santos/,"[honduras, quienopina, «Hoy más que nunca es necesaria una CICIH», Luis Javier Santos’, 2022-07-20, TEGUCIGALPA, HONDURAS.- El titular de la Unidad Fiscal Especial Contra Redes de Corrupción (Uferco), Luis Javier Santos, consideró que hoy más que nunca, se necesita que se instale la Comisión Internacional contra la Corrupción e Impunidad en Honduras (CICIH).\n\nLo anterior, luego de sumarse a las críticas por la aprobación de la Ley Especial de la Organización y Funcionamiento de la Junta Nominadora para la Proposición de Candidatos a Magistrados de la Corte Suprema de Justicia (CSJ).\n\nA...",honduras,quienopina,"«Hoy más que nunca es necesaria una CICIH», Luis Javier Santos’",2022-07-20,"TEGUCIGALPA, HONDURAS.- El titular de la Unidad Fiscal Especial Contra Redes de Corrupción (Uferco), Luis Javier Santos, consideró que hoy más que nunca, se necesita que se instale la Comisión Internacional contra la Corrupción e Impunidad en Honduras (CICIH).\n\nLo anterior, luego de sumarse a las críticas por la aprobación de la Ley Especial de la Organización y Funcionamiento de la Junta Nominadora para la Proposición de Candidatos a Magistrados de la Corte Suprema de Justicia (CSJ).\n\nAcentuó que ahora más de nunca se necesita que llegue al país una Comisión Internacional Anticorrupci..."
1,1,0,https://www.quienopina.com/category/noticias-nacionales/page/1,https://www.quienopina.com/2022/07/subsecretaria-de-seguridad-cuestiona-lentitud-de-fiscalia/,"[honduras, quienopina, Subsecretaria de Seguridad, cuestiona lentitud de Fiscalía’, 2022-07-20, TEGUCIGALPA, HONDURAS.- La subsecretaria de Seguridad, Julissa Villanueva, aseguró que las investigaciones continúan sobre la masacre suscitada en el bulevar Morazán, donde la semana pasada perdieron la vida cuatro jóvenes, entre ellos, Said Omar Lobo Bonilla, hijo del expresidente Porfirio Lobo Sosa.\n\n“Las investigaciones continúan, la carpeta de investigación no se ha detenido, sigue llenándose en los temas de criminalístico sobre todo en vaciados de teléfonos y otros trabajos periciales rel...",honduras,quienopina,"Subsecretaria de Seguridad, cuestiona lentitud de Fiscalía’",2022-07-20,"TEGUCIGALPA, HONDURAS.- La subsecretaria de Seguridad, Julissa Villanueva, aseguró que las investigaciones continúan sobre la masacre suscitada en el bulevar Morazán, donde la semana pasada perdieron la vida cuatro jóvenes, entre ellos, Said Omar Lobo Bonilla, hijo del expresidente Porfirio Lobo Sosa.\n\n“Las investigaciones continúan, la carpeta de investigación no se ha detenido, sigue llenándose en los temas de criminalístico sobre todo en vaciados de teléfonos y otros trabajos periciales relacionados con video forenses”, expresó Villanueva.\n\n“Hay una gama de acciones criminalísticas ..."
2,2,0,https://www.quienopina.com/category/noticias-nacionales/page/1,https://www.quienopina.com/2022/07/tome-califica-como-ataque-inclusion-en-lista-engel/,"[honduras, quienopina, Tomé, califica como “ataque” inclusión en «Lista Engel», 2022-07-21, TEGUCIGALPA, HONDURAS.- El vicepresidente del Congreso Nacional, Rasel Tomé, calificó como un ataque de Estados Unidos, su inclusión en la Lista Engel, aseverando que los señalamientos en su contra provienen de ataques de “grupos rabiosos de poder”.\n\nTomé recordó que desde el 2009 es objeto de persecuciones, procesos penales e informes administrativos, motivados por grupos de poder.\n\nAsimismo, condenó que su inclusión fuera por otorgar el Canal 8 al gobierno y al pueblo hondureño.\n\n“El pueblo ...",honduras,quienopina,"Tomé, califica como “ataque” inclusión en «Lista Engel»",2022-07-21,"TEGUCIGALPA, HONDURAS.- El vicepresidente del Congreso Nacional, Rasel Tomé, calificó como un ataque de Estados Unidos, su inclusión en la Lista En

In [11]:
print(len(df))
df = df[df['text'] != '']
print(len(df))
df

19382
16274


,Unnamed: 0.1,Unnamed: 0,page,link,content,country,news_outlet,title,date,text
0,0,0,https://www.quienopina.com/category/noticias-nacionales/page/1,https://www.quienopina.com/2022/07/hoy-mas-que-nunca-es-necesaria-una-cicih-luis-javier-santos/,"[honduras, quienopina, «Hoy más que nunca es necesaria una CICIH», Luis Javier Santos’, 2022-07-20, TEGUCIGALPA, HONDURAS.- El titular de la Unidad Fiscal Especial Contra Redes de Corrupción (Uferco), Luis Javier Santos, consideró que hoy más que nunca, se necesita que se instale la Comisión Internacional contra la Corrupción e Impunidad en Honduras (CICIH).\n\nLo anterior, luego de sumarse a las críticas por la aprobación de la Ley Especial de la Organización y Funcionamiento de la Junta Nominadora para la Proposición de Candidatos a Magistrados de la Corte Suprema de Justicia (CSJ).\n\nA...",honduras,quienopina,"«Hoy más que nunca es necesaria una CICIH», Luis Javier Santos’",2022-07-20,"TEGUCIGALPA, HONDURAS.- El titular de la Unidad Fiscal Especial Contra Redes de Corrupción (Uferco), Luis Javier Santos, consideró que hoy más que nunca, se necesita que se instale la Comisión Internacional contra la Corrupción e Impunidad en Honduras (CICIH).\n\nLo anterior, luego de sumarse a las críticas por la aprobación de la Ley Especial de la Organización y Funcionamiento de la Junta Nominadora para la Proposición de Candidatos a Magistrados de la Corte Suprema de Justicia (CSJ).\n\nAcentuó que ahora más de nunca se necesita que llegue al país una Comisión Internacional Anticorrupci..."
1,1,0,https://www.quienopina.com/category/noticias-nacionales/page/1,https://www.quienopina.com/2022/07/subsecretaria-de-seguridad-cuestiona-lentitud-de-fiscalia/,"[honduras, quienopina, Subsecretaria de Seguridad, cuestiona lentitud de Fiscalía’, 2022-07-20, TEGUCIGALPA, HONDURAS.- La subsecretaria de Seguridad, Julissa Villanueva, aseguró que las investigaciones continúan sobre la masacre suscitada en el bulevar Morazán, donde la semana pasada perdieron la vida cuatro jóvenes, entre ellos, Said Omar Lobo Bonilla, hijo del expresidente Porfirio Lobo Sosa.\n\n“Las investigaciones continúan, la carpeta de investigación no se ha detenido, sigue llenándose en los temas de criminalístico sobre todo en vaciados de teléfonos y otros trabajos periciales rel...",honduras,quienopina,"Subsecretaria de Seguridad, cuestiona lentitud de Fiscalía’",2022-07-20,"TEGUCIGALPA, HONDURAS.- La subsecretaria de Seguridad, Julissa Villanueva, aseguró que las investigaciones continúan sobre la masacre suscitada en el bulevar Morazán, donde la semana pasada perdieron la vida cuatro jóvenes, entre ellos, Said Omar Lobo Bonilla, hijo del expresidente Porfirio Lobo Sosa.\n\n“Las investigaciones continúan, la carpeta de investigación no se ha detenido, sigue llenándose en los temas de criminalístico sobre todo en vaciados de teléfonos y otros trabajos periciales relacionados con video forenses”, expresó Villanueva.\n\n“Hay una gama de acciones criminalísticas ..."
2,2,0,https://www.quienopina.com/category/noticias-nacionales/page/1,https://www.quienopina.com/2022/07/tome-califica-como-ataque-inclusion-en-lista-engel/,"[honduras, quienopina, Tomé, califica como “ataque” inclusión en «Lista Engel», 2022-07-21, TEGUCIGALPA, HONDURAS.- El vicepresidente del Congreso Nacional, Rasel Tomé, calificó como un ataque de Estados Unidos, su inclusión en la Lista Engel, aseverando que los señalamientos en su contra provienen de ataques de “grupos rabiosos de poder”.\n\nTomé recordó que desde el 2009 es objeto de persecuciones, procesos penales e informes administrativos, motivados por grupos de poder.\n\nAsimismo, condenó que su inclusión fuera por otorgar el Canal 8 al gobierno y al pueblo hondureño.\n\n“El pueblo ...",honduras,quienopina,"Tomé, califica como “ataque” inclusión en «Lista Engel»",2022-07-21,"TEGUCIGALPA, HONDURAS.- El vicepresidente del Congreso Nacional, Rasel Tomé, calificó como un ataque de Estados Unidos, su inclusión en la Lista En

In [12]:
if 'page' in df:
  df = df.drop(['page'], axis=1)
if 'content' in df:
  df = df.drop(['content'], axis=1)
if 'Unnamed: 0' in df:
    df = df.drop(['Unnamed: 0'], axis=1)
if 'Unnamed: 0.1' in df:
    df = df.drop(['Unnamed: 0.1'], axis=1)

In [13]:
df

,link,country,news_outlet,title,date,text
0,https://www.quienopina.com/2022/07/hoy-mas-que-nunca-es-necesaria-una-cicih-luis-javier-santos/,honduras,quienopina,"«Hoy más que nunca es necesaria una CICIH», Luis Javier Santos’",2022-07-20,"TEGUCIGALPA, HONDURAS.- El titular de la Unidad Fiscal Especial Contra Redes de Corrupción (Uferco), Luis Javier Santos, consideró que hoy más que nunca, se necesita que se instale la Comisión Internacional contra la Corrupción e Impunidad en Honduras (CICIH).\n\nLo anterior, luego de sumarse a las críticas por la aprobación de la Ley Especial de la Organización y Funcionamiento de la Junta Nominadora para la Proposición de Candidatos a Magistrados de la Corte Suprema de Justicia (CSJ).\n\nAcentuó que ahora más de nunca se necesita que llegue al país una Comisión Internacional Anticorrupci..."
1,https://www.quienopina.com/2022/07/subsecretaria-de-seguridad-cuestiona-lentitud-de-fiscalia/,honduras,quienopina,"Subsecretaria de Seguridad, cuestiona lentitud de Fiscalía’",2022-07-20,"TEGUCIGALPA, HONDURAS.- La subsecretaria de Seguridad, Julissa Villanueva, aseguró que las investigaciones continúan sobre la masacre suscitada en el bulevar Morazán, donde la semana pasada perdieron la vida cuatro jóvenes, entre ellos, Said Omar Lobo Bonilla, hijo del expresidente Porfirio Lobo Sosa.\n\n“Las investigaciones continúan, la carpeta de investigación no se ha detenido, sigue llenándose en los temas de criminalístico sobre todo en vaciados de teléfonos y otros trabajos periciales relacionados con video forenses”, expresó Villanueva.\n\n“Hay una gama de acciones criminalísticas ..."
2,https://www.quienopina.com/2022/07/tome-califica-como-ataque-inclusion-en-lista-engel/,honduras,quienopina,"Tomé, califica como “ataque” inclusión en «Lista Engel»",2022-07-21,"TEGUCIGALPA, HONDURAS.- El vicepresidente del Congreso Nacional, Rasel Tomé, calificó como un ataque de Estados Unidos, su inclusión en la Lista Engel, aseverando que los señalamientos en su contra provienen de ataques de “grupos rabiosos de poder”.\n\nTomé recordó que desde el 2009 es objeto de persecuciones, procesos penales e informes administrativos, motivados por grupos de poder.\n\nAsimismo, condenó que su inclusión fuera por otorgar el Canal 8 al gobierno y al pueblo hondureño.\n\n“El pueblo conoce nuestra hoja de vida, hay quienes se preguntan si la condena de un año de inhabilidad..."
3,https://www.quienopina.com/2022/07/ee-uu-sanciones-a-incluidos-lista-engel/,honduras,quienopina,"EE.UU, sanciones a incluidos «Lista Engel»",2022-07-21,"El director de Asuntos Centroamericanos en la Oficina del Hemisferio Occidental de Estados Unidos, Patrick Ventrell, aseguró que los señalados en la Lista Engel podrían enfrentar “acciones más permanentes”.\n\nEl funcionario se refirió en esos términos en una conferencia de prensa brindada a medios de la región, señalando que algunos de los incluidos en la Lista puede que resten importancia a su inclusión, pero recordó que Estados Unidos tiene “muchísimas más herramientas” que podría emplear en caso los señalados no cambien su comportamiento.\n\nDe igual forma, dijo que estas incluyen desd..."
4,https://www.quienopina.com/2022/07/este-es-el-estado-de-salud-de-joe-biden/,honduras,quienopina,Este es el estado de salud de «Joe Biden»,2022-07-21,"Este es el estado de salud de ""Joe Biden""\n\nEsta mañana la secretaria de prensa de la Casa Blanca, Karine Jean-Pierre, a través de un comunicado confirmó que el presidente de Estados Unidos, Joe Biden, testeó positivo para COVID-19, tras una prueba PCR de rutina.\n\nSegún medios locales, Biden tenía previsto salir de allí en las próximas dos horas hasta Pensilvania, para dar un discurso sobre el crimen y luego asistir a una recaudación de fondos en la noche de este jueves. “Está completamente vacunado y dos veces reforzado. Experimenta síntomas muy leves”, aseguraron.\n\nAsimismo, indicar..."
...,...,...,...,...,...,...
19377,https://www.quienopina.com/2014/12/honduras-a

In [14]:
df.to_csv(csv_name)

In [15]:
if(os.path.exists(output_path)):
    os.remove(output_path)
    print(('unprocessed file deleted'))

unprocessed file deleted
